In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()

REDDIT_CLIENT_ID=os.environ["REDDIT_CLIENT_ID"]
REDDIT_CLIENT_SECRET=os.environ["REDDIT_CLIENT_SECRET"] 
REDDIT_PASSWORD=os.environ["REDDIT_PASSWORD"]
REDDIT_USERNAME=os.environ["REDDIT_USERNAME"]
REDDIT_AGENT_NAME=os.environ["REDDIT_AGENT_NAME"]
SUBREDDIT_NAME=os.environ["SUBREDDIT_NAME"]
 

In [ ]:
import praw
import pandas as pd
from dataclasses import dataclass
 

reddit = praw.Reddit(
    client_id=REDDIT_CLIENT_ID,
    client_secret=REDDIT_CLIENT_SECRET,
    user_agent=REDDIT_AGENT_NAME,
    username=REDDIT_USERNAME,
    password=REDDIT_PASSWORD,
)

subreddit = reddit.subreddit(SUBREDDIT_NAME)



def fetch_new_submissions(subreddit:any, limit:int=10)->list:
    submissions = []
    for submission in subreddit.new(limit=limit):
        submissions.append(submission)
    return submissions

def fetch_hot_submissions(subreddit:any, limit:int=10)->list:
    submissions = []
    for submission in subreddit.hot(limit=limit):
        submissions.append(submission)
    return submissions

In [10]:
data = []
for post in subreddit.hot(limit=10):
 
    data.append([post.id, post.title, post.score, post.url, post.num_comments, post.selftext])

# df = pd.DataFrame(data, columns=['Id','Title', 'Score', 'URL', 'Comments', 'Text'])
# df.to_csv(f'{subreddit_name}_posts.csv', index=False)
print("Data fetched successfully!")
print(data[1])

Data fetched successfully!
['1jqlxrz', 'Zani gameplay', 6352, 'https://v.redd.it/36wtzsv65nse1', 854, '']


In [31]:
import os
import requests
import ffmpeg
from urllib.parse import urlparse, parse_qs
from praw import Reddit
from praw.models import Submission

def download_media(submission, download_dir="downloads"):
    """
    Downloads images or videos from a Reddit submission.

    Parameters:
    - submission: A PRAW Submission object.
    - download_dir: Directory where media files will be saved.
    """
    # Ensure the download directory exists
    os.makedirs(download_dir, exist_ok=True)

    # Check if the submission is a gallery (multiple images)
    if hasattr(submission, 'is_gallery') and submission.is_gallery:
        download_gallery(submission, download_dir)
    # Check if the submission is a single image
    elif submission.url.endswith(('.jpg', '.jpeg', '.png', '.gif')):
        download_file(submission.url, download_dir)
    # Check if the submission is a Reddit-hosted video
    elif submission.is_video:
        download_reddit_video(submission, download_dir)
    else:
        print("No downloadable media found in this submission.")
def download_file(url, download_dir):
    """
    Downloads a file from a given URL.

    Parameters:
    - url: The URL of the file to download.
    - download_dir: Directory where the file will be saved.
    """
    response = requests.get(url, stream=True)
    if response.status_code == 200:
        filename = os.path.join(download_dir, os.path.basename(url))
        with open(filename, 'wb') as file:
            for chunk in response.iter_content(chunk_size=8192):
                file.write(chunk)
        print(f"Downloaded: {filename}")
    else:
        print(f"Failed to download {url}")

def download_gallery(submission, download_dir):
    """
    Downloads all images from a Reddit gallery submission.

    Parameters:
    - submission: A PRAW Submission object representing a gallery.
    - download_dir: Directory where images will be saved.
    """
    if hasattr(submission, 'gallery_data') and hasattr(submission, 'media_metadata'):
        for item in submission.gallery_data['items']:
            media_id = item['media_id']
            if media_id in submission.media_metadata:
                media = submission.media_metadata[media_id]
                if 'p' in media:
                    # Get the highest resolution image
                    image_url = media['p'][-1]['u']
                    # Replace preview URL with actual image URL
                    image_url = image_url.split('?')[0].replace('preview', 'i')
                    download_file(image_url, download_dir)

def download_reddit_video(submission, download_dir):
    """
    Downloads a Reddit-hosted video and merges it with audio.

    Parameters:
    - submission: A PRAW Submission object representing a video.
    - download_dir: Directory where the video will be saved.
    """
    video_url = submission.media['reddit_video']['fallback_url']
    audio_url = video_url.rsplit('/', 1)[0] + '/DASH_audio.mp4'

    video_path = os.path.join(download_dir, f"{submission.id}_video.mp4")
    audio_path = os.path.join(download_dir, f"{submission.id}_audio.mp4")
    output_path = os.path.join(download_dir, f"{submission.id}_combined.mp4")

    # Download video and audio streams
    download_file(video_url, download_dir)
    download_file(audio_url, download_dir)

    # Merge video and audio using ffmpeg
    merge_video_audio(video_path, audio_path, output_path)

    # Clean up individual video and audio files
    os.remove(video_path)
    os.remove(audio_path)

def merge_video_audio(video_path, audio_path, output_path):
    """
    Merges video and audio files into a single file using ffmpeg.

    Parameters:
    - video_path: Path to the video file.
    - audio_path: Path to the audio file.
    - output_path: Path where the merged file will be saved.
    """
    try:
        ffmpeg.input(video_path).output(audio_path).run()
        ffmpeg.input(audio_path).output(output_path).run()
        print(f"Merged video saved as: {output_path}")
    except ffmpeg.Error as e:
        print(f"Error merging video and audio: {e}")


In [36]:
from RedDownloader import RedDownloader

subbmissions =fetch_hot_submissions(subreddit, limit=5)
 
submission = subbmissions[1]
 
video_url = submission.media['reddit_video']['fallback_url']
audio_url = video_url.replace('DASH_720', 'DASH_audio')
RedDownloader.Download(url=video_url, output='downloaded_media', quality=720)
 
 

INFO: Detected Post Type: Video
INFO: Fetching Video...
INFO: Trying resolution: 720p
INFO: Video File Downloaded Successfully
INFO: Fetching Audio...
INFO: Merging Files


Moviepy - Building video downloaded_media.mp4.
MoviePy - Writing audio in downloaded_mediaTEMP_MPY_wvf_snd.mp3


INFO: Merging Done!                                      
INFO: cleaning
INFO: downloaded_media Successfully Downloaded!
